In [1]:
class Personage:
    def __init__(
            self,
            hp,
            attack,
            defense,
            strength,
            agility,
            wisdom
    ):
        self.hp = hp
        self.attack = attack
        self.defense = defense
        self.strength = strength
        self.agility = agility
        self.wisdom = wisdom

    def advantage(self, other) -> float:
        multiplier = 2
        # Бой строится на принципе КНБ сила > ловкость > мудрость > сила
        # Считаем разницу между смежными характеристиками относительно первого персонажа
        # Например итоговая сила первого персонажа = s1 - a2 / 3 - w2
        # потом считаем разницу между получившимися характеристиками
        strength1 = max(self.strength - other.agility / multiplier - other.wisdom, 1)
        agility1 = max(self.agility - other.wisdom / multiplier - other.strength, 1)
        wisdom1 = max(self.wisdom - other.strength / multiplier - other.agility, 1)
        strength2 = max(other.strength - self.agility / multiplier - self.wisdom, 1)
        agility2 = max(other.agility - self.wisdom / multiplier - self.strength, 1)
        wisdom2 = max(other.wisdom - self.strength / multiplier - self.agility, 1)
        # print(strength1, agility1, wisdom1)
        # print(strength2, agility2, wisdom2)
        return (strength1 + agility1 + wisdom1) - (strength2 + agility2 + wisdom2)


def default_pers():
    return Personage(500, 50, 20, 5, 5, 5)


def empty_pers():
    return Personage(500, 50, 20, 1, 1, 1)


def pers_with_max_strength():
    return Personage(500, 50, 20, 13, 1, 1)


def pers_with_max_agility():
    return Personage(500, 50, 20, 1, 13, 1)


def pers_with_max_wisdom():
    return Personage(500, 50, 20, 1, 1, 13)

In [2]:
# найти максимальное преимущество
import random

repeat = 500000
max_advantage = 0
final_pers1 = None
final_pers2 = None

def increase_random_characteristic(pers: Personage):
    r = random.randrange(4)
    if r == 0:
        pers.strength += 1
    elif r == 1:
        pers.agility += 1
    else:
        pers.wisdom += 1

for _ in range(repeat):
    pers1 = empty_pers()
    pers2 = empty_pers()
    for _ in range(12):
        increase_random_characteristic(pers1)
        increase_random_characteristic(pers2)
    if abs(pers1.advantage(pers2)) > max_advantage:
        max_advantage = abs(pers1.advantage(pers2))
        final_pers1 = pers1
        final_pers2 = pers2

print(max_advantage)
print(final_pers1.strength, final_pers1.agility, final_pers1.wisdom)
print(final_pers2.strength, final_pers2.agility, final_pers2.wisdom)

5.0
11 1 3
1 1 13


In [3]:
from random import randrange
import random
import time

min_attack = 0.3

# в dodge и advantage_percent используется разновидности функции 1/x которая стремится к определённой границе,
# но никогда её не достигает

def dodge(agility1: int, agility2: int) -> bool:
    diff = agility1 - agility2
    chance = 10
    offset_x = -28
    if diff > offset_x:
        chance = chance + max(-1960 / (diff - offset_x) + 70, 0)
    result = random.randrange(100) < chance
    # print(agility1, chance, result)
    return result

def advantage_percent(advantage: float) -> float:
    return -8 / (advantage + 8) + 1

def crit(wisdom1: int, wisdom2: int) -> float:
    diff = wisdom1 - wisdom2
    chance = 10
    offset_x = -37.4
    if diff > offset_x:
        chance = chance + max(-2618 / (diff - offset_x) + 70, 0)
    multiplier = 2
    offset_multiplier_x = -6
    if diff > offset_x:
        multiplier = multiplier + max(-9 / (diff - offset_multiplier_x) + 1.5, 0)
    return multiplier if random.randrange(100) < chance else 1

def attack_multiplier(strength1: int, strength2: int) -> float:
    deviation = 0.1
    bonus = 0
    offset_x = -12
    diff = strength1 - strength2
    if diff > offset_x:
        bonus = bonus + max(-1.44 / (diff - offset_x) + 0.12, 0)
    return random.uniform(1 - deviation, 1 + deviation) + bonus

def battle(pers1: Personage, pers2: Personage) -> bool:
    # Бой строится на принципе КНБ сила > ловкость > мудрость > сила
    turn1 = randrange(2) == 0
    advantage1 = pers1.advantage(pers2)
    if advantage1 >= 0:
        bonus1 = 1 + advantage_percent(advantage1)
        bonus2 = 1
    else:
        bonus2 = 1 + advantage_percent(abs(advantage1))
        bonus1 = 1
    # print(bonus1, bonus2)
    while pers1.hp > 0 and pers2.hp > 0:
        if turn1:
            if not dodge(pers2.agility, pers1.agility):
                attack = pers1.attack \
                         * bonus1 \
                         * crit(pers1.wisdom, pers2.wisdom) \
                         * attack_multiplier(pers1.strength, pers2.strength)
                pers2.hp -= max(attack - pers2.defense, pers1.attack * min_attack)
        else:
            if not dodge(pers1.agility, pers2.agility):
                attack = pers2.attack *\
                         bonus2 *\
                         crit(pers2.wisdom, pers1.wisdom) *\
                         attack_multiplier(pers2.strength, pers1.strength)
                pers1.hp -= max(attack - pers1.defense, pers2.attack * min_attack)
        turn1 = not turn1
    # print(pers1.hp, pers2.hp)
    return pers1.hp > 0

In [4]:
repeat = 5000

In [5]:
wins = 0
min_hp = 10000
for _ in range(repeat):
    personage1 = pers_with_max_strength()
    personage2 = pers_with_max_agility()
    wins += 1 if battle(personage1, personage2) else 0
    min_hp = min(personage1.hp, min_hp)
print(wins / repeat)
# print(min_hp)

0.8254


In [6]:
wins = 0
min_hp = 10000
for _ in range(repeat):
    personage1 = pers_with_max_agility()
    personage2 = pers_with_max_wisdom()
    wins += 1 if battle(personage1, personage2) else 0
    min_hp = min(personage1.hp, min_hp)
print(wins / repeat)
# print(min_hp)

0.8214


In [7]:
wins = 0
min_hp = 10000
for _ in range(repeat):
    personage1 = pers_with_max_wisdom()
    personage2 = pers_with_max_strength()
    wins += 1 if battle(personage1, personage2) else 0
    min_hp = min(personage1.hp, min_hp)
print(wins / repeat)
# print(min_hp)

0.8286


In [8]:
wins = 0
min_hp = 10000
for _ in range(repeat):
    personage1 = Personage(500, 50, 20, 2, 5, 8)
    personage2 = Personage(500, 50, 20, 5, 5, 5)
    wins += 1 if battle(personage1, personage2) else 0
    min_hp = min(personage1.hp, min_hp)
print(wins / repeat)

0.6894


In [9]:
wins = 0
min_hp = 10000
for _ in range(repeat):
    personage1 = Personage(500, 50, 20, 9, 1, 5)
    personage2 = Personage(500, 50, 20, 5, 5, 5)
    wins += 1 if battle(personage1, personage2) else 0
    min_hp = min(personage1.hp, min_hp)
print(wins / repeat)

0.5524


In [10]:
wins = 0
min_hp = 10000
for _ in range(repeat):
    personage1 = Personage(500, 50, 20, 13, 1, 1)
    personage2 = Personage(500, 50, 20, 4, 1, 10)
    wins += 1 if battle(personage1, personage2) else 0
    min_hp = min(personage1.hp, min_hp)
print(wins / repeat)

0.462


In [11]:
p1 = Personage(500, 50, 20, 2, 5, 8)
p2 = Personage(500, 50, 20, 5, 5, 5)
print(p1.advantage(p2))

0


In [12]:
p1 = Personage(500, 50, 20, 9, 1, 5)
p2 = Personage(500, 50, 20, 5, 5, 5)
print(p1.advantage(p2))

0.5


In [13]:
p1 = Personage(500, 50, 20, 5, 9, 1)
p2 = Personage(500, 50, 20, 5, 5, 5)
print(p1.advantage(p2))

0.5


In [14]:
print(pers_with_max_agility().advantage(pers_with_max_wisdom()))

4.5
